In [1]:
import os
import sys
from pyspark.sql import SparkSession

# Spark 3.5.3 paths
spark_home = r"C:\spark\spark-3.5.3-bin-hadoop3"
sys.path.insert(0, spark_home + r"\python")
sys.path.insert(0, spark_home + r"\python\lib\py4j-0.10.9.7-src.zip")

# Python executables
os.environ["PYSPARK_PYTHON"] = r"C:\Users\Raghava\AppData\Local\Programs\Python\Python310\python.exe"
os.environ["PYSPARK_DRIVER_PYTHON"] = r"C:\Users\Raghava\AppData\Local\Programs\Python\Python310\python.exe"

In [2]:
spark = (
    SparkSession.builder
    .appName("IcebergLocalSetup-Spark353")
    .master("local[*]")
    .config(
        "spark.jars.packages",
        "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.6.1"
    )
    .config(
        "spark.sql.extensions",
        "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions"
    )
    .config(
        "spark.sql.catalog.local",
        "org.apache.iceberg.spark.SparkCatalog"
    )
    .config("spark.sql.catalog.local.type", "hadoop")
    .config(
        "spark.sql.catalog.local.warehouse",
        "file:///C:/iceberg/warehouse"
    )
    .getOrCreate()
)

In [3]:
spark.sql("SHOW CATALOGS").show()


+-------------+
|      catalog|
+-------------+
|spark_catalog|
+-------------+



In [4]:
# Create an Iceberg table
spark.sql("""
  CREATE TABLE local.schema.users (
    id INT,
    name STRING,
    age INT
  ) USING iceberg""")

# Insert some sample data
spark.sql("""
  INSERT INTO local.schema.users VALUES
    (1, 'Alice', 30),
    (2, 'Bob', 25),
    (3, 'Charlie', 35)""")

# Query the data
result = spark.sql("SELECT * FROM local.schema.users")
result.show()

+---+-------+---+
| id|   name|age|
+---+-------+---+
|  1|  Alice| 30|
|  2|    Bob| 25|
|  3|Charlie| 35|
+---+-------+---+



In [7]:
spark.sql("""
SELECT snapshot_id, committed_at
FROM local.schema.users.snapshots
ORDER BY committed_at
""").show(truncate=False)


+-------------------+-----------------------+
|snapshot_id        |committed_at           |
+-------------------+-----------------------+
|5660693612705664984|2025-12-28 17:20:47.486|
+-------------------+-----------------------+

